In [1]:
# used to widen the cells 
from IPython.core.display import display, HTML
display(HTML("<style>.container { margin-left: 2.5% !important; width:95%; }</style>"))


In [53]:
from algosdk.v2client import algod
from algosdk import mnemonic
from algosdk import transaction
import os 

algod_address = "https://testnet-algorand.api.purestake.io/ps2"
algod_token = os.environ['PURESTAKE_ALGORAND_API']
headers = {
   "X-API-Key": algod_token,
}

algod_client = algod.AlgodClient(algod_token, algod_address, headers);


In [54]:
funded_accounts = [{'sk': 'ifH/zXLAURpo8ZLVrSa7JwL46zKiCI6T8hQVDKLfl7OHJvcxU4KOymkRKGXIt9hSJYDNU4bLphvnprOMk0nXoQ==',
  'pk': 'Q4TPOMKTQKHMU2IRFBS4RN6YKISYBTKTQ3F2MG7HU2ZYZE2J26QVEUCFII',
  'memo': 'couple zone group already phone alley mercy napkin rival one talk carbon useless road bag anger impact ski media scout dune wave original absent grape'},
 {'sk': 'WUxKu75u2NQwXsc60Y3oiIn0AljrFVpHxdmTy+aqBOA+I4SYeZUnGNcbv/BEJZBslRcx0VlbrL3/t+Sb/rFaog==',
  'pk': 'HYRYJGDZSUTRRVY3X7YEIJMQNSKROMORLFN2ZPP7W7SJX7VRLKRGFM74AE',
  'memo': 'menu circle frozen typical radio cry various shrug chef dance inmate obtain trouble absent food bicycle twice bench undo rice snow click ability abstract mystery'},
 {'sk': '2FG17XNEhzK2MiqozJ+VFBqCrx4XnDpNJs9mwh9hClieyEJYxMyi2/d1QobhgArPNBzzYSyYnNoSc6gg0KRpPA==',
  'pk': 'T3EEEWGEZSRNX53VIKDODAAKZ42BZ43BFSMJZWQSOOUCBUFENE6NPBGJUM',
  'memo': 'deposit stem walnut elbow attend million noble clay never left enlist pattern aerobic program rib orchard point odor guide chapter display obscure scare able fruit'},
 {'sk': '6eC5pU1jSqJTD6LZyX1v2JDT47FLBUTiCjGR7GSMKu91Xx7H6FZtCYDH725SCUNgd7isetmt0tNVs57q9qKiUw==',
  'pk': 'OVPR5R7IKZWQTAGH55XFECKDMB33RLD23GW5FU2VWOPOV5VCUJJRI3FH74',
  'memo': 'brush travel spot crumble network trigger kind pear depth warm dash assault poet jump frown aim embrace clog obtain simple six perfect junk abstract speak'},
 {'sk': 'ApR/2KQlYK4bJR4UyOmWnhewKS5DeEXYY3OcU7Py21p8g6uBVYT5J3NxRSpSg8VmYjrlG3VeuhMaukUZlrNSKw==',
  'pk': 'PSB2XAKVQT4SO43RIUVFFA6FMZRDVZI3OVPLUEY2XJCRTFVTKIV3SKH5SQ',
  'memo': 'leopard wrestle history fog scare twist churn bullet action poet enter ketchup gasp media bonus joke anger month defy degree grit witness strategy able income'}]

In [55]:
algod_client.account_info(funded_accounts[0]['pk'])

{'address': 'Q4TPOMKTQKHMU2IRFBS4RN6YKISYBTKTQ3F2MG7HU2ZYZE2J26QVEUCFII',
 'amount': 8697000,
 'amount-without-pending-rewards': 8697000,
 'apps-local-state': [],
 'apps-total-schema': {'num-byte-slice': 3, 'num-uint': 61},
 'assets': [{'amount': 93,
   'asset-id': 13164495,
   'creator': 'WRXFDQL5GF5XVMMANEY25AQLNM4YJHYTOKRIURSD55BZAOH3G6XYHHFCWM',
   'is-frozen': False}],
 'created-apps': [{'id': 14966518,
   'params': {'approval-program': 'AiAFAAUEAYCAgICAgICAgAEmDAR2b3RlC2FkZF9vcHRpb25zB0NyZWF0b3IETmFtZQhhc3NldF9pZBFhc3NldF9jb2VmZmljaWVudBF2b3Rpbmdfc3RhcnRfdGltZQ92b3RpbmdfZW5kX3RpbWULTlVMTF9PUFRJT04Hb3B0aW9uXxRRVm90ZURlY2lzaW9uQ3JlZGl0cwEtMRgiEkAAJjEZIxJAAMkxGSQSQADHMRklEkAAxTYaACgSQADrNhoAKRJAAVwAKjEAZys2GgBnJwQ2GgYXZycFNhoHF2cnBjYaCBdnJwc2GgkXZzIHJwZkDUEAAiJDNhoBJwgTQAAFJUNCAAknCTYaAVAhBGc2GgInCBNAAAUlQ0IACScJNhoCUCEEZzYaAycIE0AABSVDQgAJJwk2GgNQIQRnNhoEJwgTQAAFJUNCAAknCTYaBFAhBGc2GgUnCBNAAAUlQ0IACScJNhoFUCEEZyVDJUNCATEiQ0IBLCJDQgEnMgcnBmQNQQACIkMiJwRkcAA1ADUBNABAAAUiQ0IADCInCjQBJ

In [56]:
def compile(approval_filename='approval.teal', clear_state_filename='clear_state.teal'):

    # pyteal to teal 
    with open(approval_filename, 'w') as f:
        compiled = compileTeal(approval_program(), Mode.Application)
        f.write(compiled)

    with open(clear_state_filename, 'w') as f:
        compiled = compileTeal(clear_state_program(), Mode.Application)
        f.write(compiled)

    # teal to bytecode 
    stdout, stderr = execute(["goal", "clerk", "compile", "-o", approval_filename+'.tok', approval_filename])
    if stderr != "":
        print(stderr)
        raise
    elif len(stdout) < 59:
        print("error in compile teal")
        raise

    stdout, stderr = execute(["goal", "clerk", "compile", "-o", clear_state_filename+'.tok', clear_state_filename])
    if stderr != "":
        print(stderr)
        raise
    elif len(stdout) < 59:
        print("error in compile teal")
        raise

    with open(approval_filename+'.tok', 'rb') as f:
        approval_bytes = f.read()

    with open(clear_state_filename+'.tok', 'rb') as f: 
        clear_state_bytes = f.read() 
        
    return approval_bytes, clear_state_bytes

In [57]:
# read app global state
def read_global_state(client, addr, app_id):   
    results = client.account_info(addr)
    apps_created = results['created-apps']
    global_states = []
    for app in apps_created :
        if app['id'] == app_id :
            global_states.append(app['params']['global-state'])
            # print(f"global_state for app_id {app_id}: ", app['params']['global-state'])
    return global_states 

In [58]:
import base64 
def decode_keys(s):
    for entry in s[0]:
        entry['key'] = base64.b64decode(entry['key']).decode('utf-8')
    return s

In [59]:
import time

In [60]:
from pyteal import *    
    
def approval_program():     
    VOTING_CREDIT_SYM = Bytes("QVoteDecisionCredits")
    OPTION_PREFIX = Bytes("option_")
    NULL_OPTION = Bytes("NULL_OPTION")
    ZERO = Int(2**63)     # to have neative numbers 
    MINUS = Bytes("-")
    
    arg_num = Txn.application_args.length()
    
    on_closeout = Return(Int(1))
    
    asset_id = App.globalGet(Bytes("asset_id"))
    asset_balance = AssetHolding.balance(Int(0), asset_id) 
    asset_coeff = App.globalGet(Bytes("asset_coefficient"))
    on_optin = Seq([
        If(Global.latest_timestamp() > App.globalGet(Bytes("voting_start_time")), 
          Return(Int(0))),
        asset_balance, 
        If(asset_balance.hasValue(),  # if sender has some tokens, store it, otherwise return 0     
           Seq([App.localPut(Int(0), VOTING_CREDIT_SYM, Mul(asset_balance.value(), asset_coeff)), Return(Int(1))]),      
           Return(Int(0))
        ) 
    ])  
    
    # TODO registration time 
    option0 = Concat(OPTION_PREFIX, Txn.application_args[1])
    option0_tally = App.globalGetEx(Int(0), option0)            # current total of votes for this option
    option0_votes = Btoi(Txn.application_args[2])               # how much the user wants to vote for this option 
    option0_votes_sign = Txn.application_args[3]
    credit_balance = App.localGet(Int(0), VOTING_CREDIT_SYM)
    on_vote = Seq([ 
        If(Global.latest_timestamp() < App.globalGet(Bytes("voting_start_time")), 
          Return(Int(0))),
        If(Global.latest_timestamp() > App.globalGet(Bytes("voting_end_time")), 
          Return(Int(0))),
        option0_tally, 
        If(option0_tally.hasValue(),      # if user chose a valid option 
           Seq([
               App.localPut(Int(0), VOTING_CREDIT_SYM, credit_balance-Mul(option0_votes, option0_votes)),     # update balance
               If(Ge(credit_balance, Int(0)),             # does credit_balance this get re-evaluated? 
                    If(option0_votes_sign == MINUS,
                       App.globalPut(option0, option0_tally.value()-option0_votes),    
                       App.globalPut(option0, option0_tally.value()+option0_votes), 
                      ),
                    Return(Int(0))
                 ),    
               Return(Int(1))
           ]),
           Return(Int(0))
          )
    ])
    
    
    # add up to 5 options at a time. 
    on_add_options =  Seq([
        If(Global.latest_timestamp() > App.globalGet(Bytes("voting_start_time")), 
          Return(Int(0))),
        If(Txn.application_args[1] != NULL_OPTION, 
           Seq([App.globalPut(Concat(OPTION_PREFIX, Txn.application_args[1]), ZERO)]),
           Return(Int(1))
        ),
        If(Txn.application_args[2] != NULL_OPTION, 
          Seq([App.globalPut(Concat(OPTION_PREFIX, Txn.application_args[2]), ZERO)]),
           Return(Int(1))
        ),
        If(Txn.application_args[3] != NULL_OPTION, 
          Seq([App.globalPut(Concat(OPTION_PREFIX, Txn.application_args[3]), ZERO)]),
           Return(Int(1))
        ),
        If(Txn.application_args[4] != NULL_OPTION, 
          Seq([App.globalPut(Concat(OPTION_PREFIX, Txn.application_args[4]), ZERO)]),
           Return(Int(1))
        ),
        If(Txn.application_args[5] != NULL_OPTION, 
          Seq([App.globalPut(Concat(OPTION_PREFIX, Txn.application_args[5]), ZERO)]),
           Return(Int(1))
        ), 
        Return(Int(1))
    ])
    
    
    on_creation = Seq([    
        App.globalPut(Bytes("Creator"), Txn.sender()),    
        App.globalPut(Bytes("Name"), Txn.application_args[0]),
        App.globalPut(Bytes("asset_id"), Btoi(Txn.application_args[6])), 
        App.globalPut(Bytes("asset_coefficient"), Btoi(Txn.application_args[7])), 
        App.globalPut(Bytes("voting_start_time"), Btoi(Txn.application_args[8])),
        App.globalPut(Bytes("voting_end_time"), Btoi(Txn.application_args[9])),
        on_add_options,   # record the options 
        Return(Int(1))
    ])    
    
        
    program = Cond(    
            [Txn.application_id() == Int(0), on_creation],    
            [Txn.on_completion() == OnComplete.DeleteApplication, Return(Int(0))], 
            [Txn.on_completion() == OnComplete.UpdateApplication, Return(Int(0))],    
            [Txn.on_completion() == OnComplete.OptIn, on_optin],
            [Txn.application_args[0] == Bytes("vote"), on_vote],
            [Txn.application_args[0] == Bytes("add_options"), on_add_options]
    )    
    return program    
    
    
def clear_state_program():     
    program = Return(Int(1))
    return program    


## Deploy

In [61]:
from algosdk.future import transaction
from algosdk.future.transaction import StateSchema
from algosdk.future.transaction import OnComplete as onComplete

In [62]:
params = algod_client.suggested_params()
params.falt_fee = True
params.fee = 1000

In [65]:
asset_id = 13164495    # this asset is used to compare 
asset_coefficient = 2    # how many voting coins you get for each token of the asset you own 

approval_bytes, clear_state_bytes = compile()

decision_name = 'muchdecision'

local_schema = StateSchema(num_uints=1, num_byte_slices=1)    
global_schema = StateSchema(num_uints=61, num_byte_slices=3)     # maximum sum is 64

registration_seconds = 150
voting_seconds = 300

start_time = round(time.time()) + registration_seconds
end_time = start_time + voting_seconds

on_complete = onComplete(0)
app_create_txn = transaction.ApplicationCreateTxn(
    funded_accounts[0]['pk'], 
    params, 
    on_complete, 
    approval_bytes, 
    clear_state_bytes, 
    global_schema, 
    local_schema,
    # you always need to submit this many options. you can use NULL_OPTION to ingore an option 
    app_args = [decision_name.encode('utf-8'), 
                "first".encode('utf-8'),
                "second".encode('utf-8'),
                "third".encode('utf-8'), 
                "NULL_OPTION".encode('utf-8'),
                "NULL_OPTION".encode('utf-8'), 
                asset_id.to_bytes(3, 'big'),
                asset_coefficient.to_bytes(2, 'big'), 
                start_time.to_bytes(6, 'big'),
                end_time.to_bytes(6, 'big')]
)

app_create_txn_signed = app_create_txn.sign(funded_accounts[0]['sk'])
txid = algod_client.send_transaction(app_create_txn_signed)
txid

'CAV4Q73G575OTISGSOZRUS5DBJHIUAWBGKCK7HYIYVT4QEA67SWQ'

In [87]:
round(time.time()) - start_time

1

In [69]:
app_id = algod_client.pending_transaction_info(txid)['application-index']
app_id

14967091

In [71]:
global_state = read_global_state(algod_client, funded_accounts[0]['pk'], app_id)
decode_keys(global_state)

[[{'key': 'voting_end_time',
   'value': {'bytes': '', 'type': 2, 'uint': 1616512681}},
  {'key': 'Creator',
   'value': {'bytes': 'hyb3MVOCjsppEShlyLfYUiWAzVOGy6Yb56azjJNJ16E=',
    'type': 1,
    'uint': 0}},
  {'key': 'asset_id', 'value': {'bytes': '', 'type': 2, 'uint': 13164495}},
  {'key': 'option_first',
   'value': {'bytes': '', 'type': 2, 'uint': 9223372036854775808}},
  {'key': 'option_second',
   'value': {'bytes': '', 'type': 2, 'uint': 9223372036854775808}},
  {'key': 'Name',
   'value': {'bytes': 'bXVjaGRlY2lzaW9u', 'type': 1, 'uint': 0}},
  {'key': 'asset_coefficient', 'value': {'bytes': '', 'type': 2, 'uint': 2}},
  {'key': 'voting_start_time',
   'value': {'bytes': '', 'type': 2, 'uint': 1616512381}},
  {'key': 'option_third',
   'value': {'bytes': '', 'type': 2, 'uint': 9223372036854775808}}]]

## Add more options

In [72]:
def add_options_tx(option_names, sender=0):
    app_args = ["add_options".encode("utf-8")] + [option_name.encode('utf-8') for option_name in option_names]

    # create unsigned transaction
    call_tx = transaction.ApplicationNoOpTxn(funded_accounts[sender]['pk'], params, app_id, app_args)
    call_txid = algod_client.send_transaction(call_tx.sign(funded_accounts[sender]['sk']))
    return call_txid

call_txid = add_options_tx(["new_one", "new_two", "new_three", "new_four", "NULL_OPTION"])

In [74]:
global_state = decode_keys(read_global_state(algod_client, funded_accounts[0]['pk'], app_id))
global_state

[[{'key': 'asset_coefficient', 'value': {'bytes': '', 'type': 2, 'uint': 2}},
  {'key': 'Name',
   'value': {'bytes': 'bXVjaGRlY2lzaW9u', 'type': 1, 'uint': 0}},
  {'key': 'voting_start_time',
   'value': {'bytes': '', 'type': 2, 'uint': 1616512381}},
  {'key': 'voting_end_time',
   'value': {'bytes': '', 'type': 2, 'uint': 1616512681}},
  {'key': 'option_first',
   'value': {'bytes': '', 'type': 2, 'uint': 9223372036854775808}},
  {'key': 'Creator',
   'value': {'bytes': 'hyb3MVOCjsppEShlyLfYUiWAzVOGy6Yb56azjJNJ16E=',
    'type': 1,
    'uint': 0}},
  {'key': 'option_second',
   'value': {'bytes': '', 'type': 2, 'uint': 9223372036854775808}},
  {'key': 'option_third',
   'value': {'bytes': '', 'type': 2, 'uint': 9223372036854775808}},
  {'key': 'asset_id', 'value': {'bytes': '', 'type': 2, 'uint': 13164495}}]]

## Opt-in

In [75]:
optin_tx = transaction.ApplicationOptInTxn(funded_accounts[1]['pk'], params, app_id)
algod_client.send_transaction(optin_tx.sign(funded_accounts[1]['sk']))

'AAOMYWTJUL3Y27CT6YDAZPO74VY2M6QEVF5XDPDBR5Z7KQPPE3KA'

In [88]:
optin_tx = transaction.ApplicationOptInTxn(funded_accounts[3]['pk'], params, app_id)
algod_client.send_transaction(optin_tx.sign(funded_accounts[3]['sk']))

optin_tx = transaction.ApplicationOptInTxn(funded_accounts[4]['pk'], params, app_id)
algod_client.send_transaction(optin_tx.sign(funded_accounts[4]['sk']))

AlgodHTTPError: {"message":"TransactionPool.Remember: transaction VQLM2IAPQ5QYMPPZLMSOGVEKDD6DKMJHV3KDVWZUZ5CY4YSKBQ4A: transaction rejected by ApprovalProgram"}


In [77]:
local_storage = algod_client.account_info(funded_accounts[1]['pk'])['apps-local-state']
local_storage

[{'id': 14966518,
  'key-value': [{'key': 'UVZvdGVEZWNpc2lvbkNyZWRpdHM=',
    'value': {'bytes': '', 'type': 2, 'uint': 50}}],
  'schema': {'num-byte-slice': 1, 'num-uint': 1}},
 {'id': 14967091,
  'key-value': [{'key': 'UVZvdGVEZWNpc2lvbkNyZWRpdHM=',
    'value': {'bytes': '', 'type': 2, 'uint': 84}}],
  'schema': {'num-byte-slice': 1, 'num-uint': 1}}]

In [123]:
local_storage = algod_client.account_info(funded_accounts[3]['pk'])['apps-local-state']
local_storage

[{'id': 14959763,
  'key-value': [{'key': 'UVZvdGVEZWNpc2lvbkNyZWRpdHM=',
    'value': {'bytes': '', 'type': 2, 'uint': 128}}],
  'schema': {'num-byte-slice': 1, 'num-uint': 1}},
 {'id': 14962264,
  'key-value': [{'key': 'UVZvdGVEZWNpc2lvbkNyZWRpdHM=',
    'value': {'bytes': '', 'type': 2, 'uint': 128}}],
  'schema': {'num-byte-slice': 1, 'num-uint': 1}},
 {'id': 14962278,
  'key-value': [{'key': 'UVZvdGVEZWNpc2lvbkNyZWRpdHM=',
    'value': {'bytes': '', 'type': 2, 'uint': 128}}],
  'schema': {'num-byte-slice': 1, 'num-uint': 1}},
 {'id': 14707630,
  'key-value': [{'key': 'UVZvdGVEZWNpc2lvbkNyZWRpdHM=',
    'value': {'bytes': '', 'type': 2, 'uint': 128}}],
  'schema': {'num-byte-slice': 1, 'num-uint': 1}}]

In [32]:
local_storage = algod_client.account_info(funded_accounts[4]['pk'])['apps-local-state']
local_storage

[{'id': 14959763,
  'key-value': [{'key': 'UVZvdGVEZWNpc2lvbkNyZWRpdHM=',
    'value': {'bytes': '', 'type': 2, 'uint': 186}}],
  'schema': {'num-byte-slice': 1, 'num-uint': 1}},
 {'id': 14707630,
  'key-value': [{'key': 'UVZvdGVEZWNpc2lvbkNyZWRpdHM=',
    'value': {'bytes': '', 'type': 2, 'uint': 186}}],
  'schema': {'num-byte-slice': 1, 'num-uint': 1}}]

In [33]:
# if we look at the asset balances of these accounts we can check that the voting credits are asset_coefficient * asset_balance 
# account 1 has already opted in too many apps, that's why it isn't here

In [34]:
for acc in funded_accounts:
    print(algod_client.account_info(acc['pk'])['assets'])

[]
[{'amount': 42, 'asset-id': 13164495, 'creator': 'WRXFDQL5GF5XVMMANEY25AQLNM4YJHYTOKRIURSD55BZAOH3G6XYHHFCWM', 'is-frozen': False}]
[{'amount': 73, 'asset-id': 13164495, 'creator': 'WRXFDQL5GF5XVMMANEY25AQLNM4YJHYTOKRIURSD55BZAOH3G6XYHHFCWM', 'is-frozen': False}]
[{'amount': 64, 'asset-id': 13164495, 'creator': 'WRXFDQL5GF5XVMMANEY25AQLNM4YJHYTOKRIURSD55BZAOH3G6XYHHFCWM', 'is-frozen': False}]
[{'amount': 93, 'asset-id': 13164495, 'creator': 'WRXFDQL5GF5XVMMANEY25AQLNM4YJHYTOKRIURSD55BZAOH3G6XYHHFCWM', 'is-frozen': False}]


In [42]:
# let's try opting in an account with no assets 
optin_tx = transaction.ApplicationOptInTxn(funded_accounts[0]['pk'], params, app_id)
algod_client.send_transaction(optin_tx.sign(funded_accounts[0]['sk']))
# this tx is rejected by the approval program 

AlgodHTTPError: {"message":"TransactionPool.Remember: transaction N3XCGIKJPRNOCUXH3GS5Q5LNTNKWYRZA5UILGLH6XOCO4UIBQNIQ: transaction rejected by ApprovalProgram"}


## Voting

In [38]:
## negative vote 

In [89]:
option_name = "first"
votes = 3
sign = "-"
app_args = ["vote".encode("utf-8"), option_name.encode("utf-8"), votes.to_bytes(2, "big"), sign.encode("utf-8")]

# create unsigned transaction
call_tx = transaction.ApplicationNoOpTxn(funded_accounts[1]['pk'], params, app_id, app_args)
call_txid = algod_client.send_transaction(call_tx.sign(funded_accounts[1]['sk']))

In [92]:
# balance has gone down by the square of the amount 
local_storage = algod_client.account_info(funded_accounts[1]['pk'])['apps-local-state']
local_storage

[{'id': 14966518,
  'key-value': [{'key': 'UVZvdGVEZWNpc2lvbkNyZWRpdHM=',
    'value': {'bytes': '', 'type': 2, 'uint': 50}}],
  'schema': {'num-byte-slice': 1, 'num-uint': 1}},
 {'id': 14967091,
  'key-value': [{'key': 'UVZvdGVEZWNpc2lvbkNyZWRpdHM=',
    'value': {'bytes': '', 'type': 2, 'uint': 75}}],
  'schema': {'num-byte-slice': 1, 'num-uint': 1}}]

In [93]:
# vote tally has gone up by the right amount 
global_state = decode_keys(read_global_state(algod_client, funded_accounts[0]['pk'], app_id))
global_state

[[{'key': 'option_new_three',
   'value': {'bytes': '', 'type': 2, 'uint': 9223372036854775808}},
  {'key': 'Creator',
   'value': {'bytes': 'hyb3MVOCjsppEShlyLfYUiWAzVOGy6Yb56azjJNJ16E=',
    'type': 1,
    'uint': 0}},
  {'key': 'option_second',
   'value': {'bytes': '', 'type': 2, 'uint': 9223372036854775808}},
  {'key': 'option_third',
   'value': {'bytes': '', 'type': 2, 'uint': 9223372036854775808}},
  {'key': 'option_new_two',
   'value': {'bytes': '', 'type': 2, 'uint': 9223372036854775808}},
  {'key': 'Name',
   'value': {'bytes': 'bXVjaGRlY2lzaW9u', 'type': 1, 'uint': 0}},
  {'key': 'asset_coefficient', 'value': {'bytes': '', 'type': 2, 'uint': 2}},
  {'key': 'asset_id', 'value': {'bytes': '', 'type': 2, 'uint': 13164495}},
  {'key': 'voting_start_time',
   'value': {'bytes': '', 'type': 2, 'uint': 1616512381}},
  {'key': 'option_new_one',
   'value': {'bytes': '', 'type': 2, 'uint': 9223372036854775808}},
  {'key': 'voting_end_time',
   'value': {'bytes': '', 'type': 2, 'uin

In [47]:
# TODO 2 decimal places - this should actually be handled by the sdk together with the asset_coefficient 

In [39]:
## positive vote 

In [94]:
option_name = "first"
votes = 5
sign = "+"
app_args = ["vote".encode("utf-8"), option_name.encode("utf-8"), votes.to_bytes(2, "big"), sign.encode("utf-8")]

# create unsigned transaction
call_tx = transaction.ApplicationNoOpTxn(funded_accounts[1]['pk'], params, app_id, app_args)
call_txid = algod_client.send_transaction(call_tx.sign(funded_accounts[1]['sk']))

In [96]:
# balance has gone down by the square of the amount 
local_storage = algod_client.account_info(funded_accounts[1]['pk'])['apps-local-state']
local_storage

[{'id': 14967091,
  'key-value': [{'key': 'UVZvdGVEZWNpc2lvbkNyZWRpdHM=',
    'value': {'bytes': '', 'type': 2, 'uint': 50}}],
  'schema': {'num-byte-slice': 1, 'num-uint': 1}},
 {'id': 14966518,
  'key-value': [{'key': 'UVZvdGVEZWNpc2lvbkNyZWRpdHM=',
    'value': {'bytes': '', 'type': 2, 'uint': 50}}],
  'schema': {'num-byte-slice': 1, 'num-uint': 1}}]

In [97]:
# vote tally has gone up by the right amount 
global_state = decode_keys(read_global_state(algod_client, funded_accounts[0]['pk'], app_id))
global_state

[[{'key': 'option_new_three',
   'value': {'bytes': '', 'type': 2, 'uint': 9223372036854775808}},
  {'key': 'Creator',
   'value': {'bytes': 'hyb3MVOCjsppEShlyLfYUiWAzVOGy6Yb56azjJNJ16E=',
    'type': 1,
    'uint': 0}},
  {'key': 'voting_end_time',
   'value': {'bytes': '', 'type': 2, 'uint': 1616512681}},
  {'key': 'option_new_four',
   'value': {'bytes': '', 'type': 2, 'uint': 9223372036854775808}},
  {'key': 'asset_id', 'value': {'bytes': '', 'type': 2, 'uint': 13164495}},
  {'key': 'voting_start_time',
   'value': {'bytes': '', 'type': 2, 'uint': 1616512381}},
  {'key': 'Name',
   'value': {'bytes': 'bXVjaGRlY2lzaW9u', 'type': 1, 'uint': 0}},
  {'key': 'asset_coefficient', 'value': {'bytes': '', 'type': 2, 'uint': 2}},
  {'key': 'option_first',
   'value': {'bytes': '', 'type': 2, 'uint': 9223372036854775810}},
  {'key': 'option_third',
   'value': {'bytes': '', 'type': 2, 'uint': 9223372036854775808}},
  {'key': 'option_new_two',
   'value': {'bytes': '', 'type': 2, 'uint': 92233